In [ ]:
import random
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def generate_erdos_renyi_graphs(n, p, num_graphs):
    graphs = []
    for i in range(num_graphs):
        graphs.append(nx.erdos_renyi_graph(n, p))
    return graphs

def generate_barabasi_albert_graphs(n, m, num_graphs):
    graphs = []
    for i in range(num_graphs):
        graphs.append(nx.barabasi_albert_graph(n, m))
    return graphs

def generate_watts_strogatz_graphs(n, k, p, num_graphs):
    graphs = []
    for i in range(num_graphs): #k yi boris veriyor hukum
        graphs.append(nx.watts_strogatz_graph(n, k, p))
    return graphs

def generate_random_regular_graphs(d, n, num_graphs):
    graphs = []
    for i in range(num_graphs):
        graphs.append(nx.random_regular_graph(d, n))
    return graphs

def generate_powerlaw_cluster_graphs(n, m, p, num_graphs):
    graphs = []
    for i in range(num_graphs):
        graphs.append(nx.powerlaw_cluster_graph(n, m, p))
    return graphs

def generate_random_geometric_graphs(n, r, num_graphs):
    graphs = []
    for i in range(num_graphs):
        graphs.append(nx.random_geometric_graph(n, r))
    return graphs

# Adjacency Matrix
def graph_to_adj_matrix(G):
    return nx.to_numpy_array(G)

In [ ]:
# create different types of graphs in a for loop
graphs = []

num_graphs = 1000

n = 100
p = 0.0075
graphs += generate_erdos_renyi_graphs(n, p, num_graphs)

n = 100
k = 3
p = 0.0001
graphs += generate_watts_strogatz_graphs(n, k, p, num_graphs)

n = 100
m = 2
for i in range(num_graphs):
  k = random.randint(2, n/2)
  p = random.uniform(0.00001, 0.5)
  graphs += nx.barabasi_albert_graph(n, m)
graphs = generate_watts_strogatz_graphs (100, 4, 0.000000000000001, 1000)

In [ ]:
connected_graphs = [graph for graph in graphs if nx.is_connected(graph)]

In [ ]:
# create a dataframe to store the results
df = pd.DataFrame(columns=['graph_type', 'data', 'num_nodes', 'num_edges', 'label'])

# calculate the metrics for each graph
for graph in connected_graphs:
    graph_type = graph.__class__.__name__
    data = graph_to_adj_matrix(graph)
    num_nodes = graph.number_of_nodes()
    num_edges = graph.number_of_edges()

    Commenting out the calculations not needed for CNN
    avg_degree = np.mean([deg for node, deg in graph.degree()])
    avg_clustering_coefficient = nx.average_clustering(graph)
    if nx.is_connected(graph):
        avg_shortest_path_length = nx.average_shortest_path_length(graph)
        avg_eccentricity = np.mean([ecc for node, ecc in nx.eccentricity(graph).items()])
        avg_diameter = nx.diameter(graph)
        avg_radius = nx.radius(graph)
    else:
        avg_shortest_path_length = None  # or a default value
        avg_eccentricity = None  # or a default value
        avg_diameter = None  # or a default value
        avg_radius = None  # or a default value
    avg_density = nx.density(graph)
    avg_closeness_centrality = np.mean([cc for node, cc in nx.closeness_centrality(graph).items()])
    avg_betweenness_centrality = np.mean([bc for node, bc in nx.betweenness_centrality(graph).items()])
    avg_eigenvector_centrality = np.mean([ec for node, ec in nx.eigenvector_centrality(graph).items()])
    avg_pagerank = np.mean([pr for node, pr in nx.pagerank(graph).items()])

    try:
        partition_first, partition_second = nx.algorithms.community.kernighan_lin_bisection(graph)
        graph_first = graph.subgraph(partition_first)
        graph_second = graph.subgraph(partition_second)

        # Check if the partition divides the graph into nearly equal halves
        if abs(len(partition_first) - len(partition_second)) <= 0 and nx.is_connected(graph_first) and nx.is_connected(graph_second):
            label = 'Yes'
        else:
            label = 'No'
    except:
        print('Error')
        pass

    # Append only the necessary data for CNN
    df.loc[len(df)] = [graph_type, data, num_nodes, num_edges, label]

In [ ]:
# Display the total number of graphs
total_graphs = len(df)
print(f"Total number of graphs: {total_graphs}")

# Display the count of each label
label_counts = df['label'].value_counts()
print("Label counts:")
print(label_counts)

Total number of graphs: 1000
Label counts:
Yes    703
No     297
Name: label, dtype: int64


In [ ]:
df.tail()

In [ ]:
# Step 2: Data Preprocessing

# Find the maximum size of the graph (max number of nodes)
max_size = max(df['data'].apply(lambda x: x.shape[0]))

# Pad the adjacency matrices with zeros
X_pre = [np.pad(matrix, ((0, max_size - matrix.shape[0]), (0, max_size - matrix.shape[1])), 'constant') for matrix in df['data']]

# Convert to NumPy array
X_pre = np.array(X_pre, dtype=np.float32)

# Convert labels from 'Yes'/'No' to 1/0
y_pre = df['label'].values
y = np.array([1 if label == 'Yes' else 0 for label in y_pre])

# Now X_pre is your input data
X = X_pre


In [ ]:
# Step 3: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 4: Model Architecture
model = Sequential()
model.add(Reshape((n, n, 1), input_shape=(n, n)))  # Reshape for Conv2D
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
# Step 5: Model Compilation
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Step 6: Model Training
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1)

In [ ]:
# Step 7: Model Evaluation
accuracy = model.evaluate(X_test, y_test)[1]
print(f'Test Accuracy: {accuracy}')